In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
from sklearn.decomposition.nmf import non_negative_factorization
import pandas as pd
import numpy as np
import scanpy as sc

/opt/miniconda3/envs/seqwell/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.decomposition.nmf module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/miniconda3/envs/seqwell/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [10]:
counts_adata_fn = 'example_PBMC/counts.h5ad'
adata = sc.read(counts_adata_fn)

In [11]:
counts = pd.DataFrame(adata.X.todense(), index=adata.obs.index, columns=adata.var.index)
counts.head()

,AL627309.1,AP006222.2,RP11-206L10.2,RP11-206L10.9,LINC00115,NOC2L,KLHL17,PLEKHN1,RP11-54O7.17,HES4,...,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AC145212.1,AL592183.1,AL354822.1,PNRC2-1,SRSF10-1
AAACATACAACCAC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
AAACATTGAGCTAC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,33.0,1.0,0.0,8.0,0.0,1.0,0.0,0.0,0.0
AAACATTGATCAGC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
AAACCGTGCTTCCG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
AAACCGTGTATGCG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [15]:
spectrafn = 'example_PBMC/cNMF/pbmc_cNMF/pbmc_cNMF.gene_spectra_tpm.k_8.dt_0_15.txt' # you want the TPM spectra file with the appropriate outlier filtering parameter
spectra = pd.read_csv(spectrafn, sep='\t', index_col=0)
spectra.head()

,AL627309.1,AP006222.2,RP11-206L10.2,RP11-206L10.9,LINC00115,NOC2L,KLHL17,PLEKHN1,RP11-54O7.17,HES4,...,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AC145212.1,AL592183.1,AL354822.1,PNRC2-1,SRSF10-1
1,0.513054,0.451266,0.312546,0.000000,1.722371,39.976065,0.571123,0.363457,0.000000,0.000000,...,73.955920,2526.746828,378.070998,42.830979,2253.666498,0.000000,58.499230,0.295602,24.643361,13.333777
2,0.000000,7.190319,0.000000,0.000000,0.000000,0.000000,52.676908,0.000000,0.003797,8.514591,...,0.000000,683.903467,20.456062,0.000000,1176.752617,0.000000,39.394277,1.416284,4.509083,0.000000
3,0.000000,0.000000,2.103115,0.000000,6.919779,105.646103,0.000000,2.025341,0.000000,0.000000,...,40.256276,686.969064,504.047608,60.034579,0.000000,24.310334,109.556352,3.114976,0.000000,6.174641
4,7.365699,1.141755,0.025645,1.347231,2.744350,24.195885,1.215814,4.997773,2.277592,0.000000,...,81.710307,1685.602489,407.039886,39.344231,2081.406425,3.186238,35.781724,0.000000,6.480940,0.000000
5,1.524331,0.257749,0.000000,0.000000,4.909336,68.321285,7.699949,2.912993,0.000000,0.000000,...,91.315508,2374.853040,411.548397,63.450104,1992.437410,0.000000,71.315958,0.000000,20.610764,10.139343


In [22]:
## Can optionally filter HVGs
hvgfn = 'example_PBMC/cNMF/pbmc_cNMF/pbmc_cNMF.overdispersed_genes.txt'
hvgs = open(hvgfn).read().split('\n')
hvgs[:5]

['ISG15', 'CPSF3L', 'MRPL20', 'ATAD3C', 'C1orf86']

In [24]:
## By default the spectra are pretty close to each other and sum to pretty close to 1 million
spectra.sum(axis=1)

1    1.004791e+06
2    1.136506e+06
3    1.045571e+06
4    1.002282e+06
5    1.001869e+06
6    1.009562e+06
7    1.008673e+06
8    1.063568e+06
dtype: float64

In [26]:
## But we still normalize to sum to 1 to make sure usage coefficients are on the same scale
spectra_norm = spectra.div(spectra.sum(axis=1), axis=0)
spectra_norm.sum(axis=1)

1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    1.0
8    1.0
dtype: float64

In [27]:
def _nmf(X, nmf_kwargs, topic_labels=None):
        """
        Parameters
        ----------
        X : pandas.DataFrame,
            Normalized counts dataFrame to be factorized.

        nmf_kwargs : dict,
            Arguments to be passed to ``non_negative_factorization``

        """
        (W, H, niter) = non_negative_factorization(X.values, **nmf_kwargs)

        usages = pd.DataFrame(W, index=X.index, columns=topic_labels)
        spectra = pd.DataFrame(H, columns=X.columns, index=topic_labels)
        return spectra, usages
    
    
refit_nmf_kwargs = dict(
            n_components = spectra_norm.shape[0],
            H = spectra_norm.values,
            update_H = False,
            shuffle = True,

            alpha=0.0,
            l1_ratio=0.0,
            beta_loss='kullback-leibler',
            solver='mu',
            tol=1e-4,
            max_iter=1000,
            regularization=None,
        )

(s,usage) = _nmf(counts, refit_nmf_kwargs, topic_labels=spectra_norm.index)

/opt/miniconda3/envs/seqwell/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:1035: FutureWarning: The default value of init will change from random to None in 0.23 to make it consistent with decomposition.NMF.
  "with decomposition.NMF.", FutureWarning)


In [31]:
## These are the refit usages
usage.head()

,1,2,3,4,5,6,7,8
AAACATACAACCAC-1,1994.111454,3.096449,109.968573,40.737472,210.204078,13.324579,47.481647,0.075747
AAACATTGAGCTAC-1,1719.461865,0.406315,504.465153,28.696978,66.489115,2413.426342,170.008081,0.046151
AAACATTGATCAGC-1,2236.707117,1.144199,389.697425,21.804884,286.019904,8.727328,181.616503,21.282641
AAACCGTGCTTCCG-1,149.058238,0.325103,95.631642,905.648367,18.090375,184.674189,1256.886141,28.685946
AAACCGTGTATGCG-1,15.980768,0.003072,54.104124,84.906783,793.399477,2.784121,28.821651,0.000005


In [32]:
## Usually we want normalized usages
usage_norm = usage.div(usage.sum(axis=1), axis=0)